# Forecasting economic activity

## Combinining datasets, adding lags

In [1]:
# Load packages
#install.packages("lattice")
#install.packages("caret")
#install.packages("Hmisc")
library(readxl)
library(zoo)
library(tidyverse)
library(fastDummies)
library(caret)
library(rlang)
library(Hmisc)


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift



Attaching package: ‘rlang’


The following objects are masked from ‘package:purrr’:

    %@%, as_function, flatten, flatten_chr, flatten_dbl, flatten_int,
    flatten_lgl, flatten_raw, invoke, splice


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Form

In [2]:
# Load news data
load('../../text_analysis/data/news_variables.Rdata')
head(news_variables)

,month,gamma_1,gamma_2,gamma_3,gamma_4,gamma_5,gamma_6,gamma_7,gamma_8,count_1,⋯,news_5,news_6,news_7,news_8,pos_dum,neg_dum,index,var_index,var_pos,var_neg
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2012.833,0.05840190,0.3212144,0.1341846,0.0006955618,0.08852647,0.12119587,0.06646040,0.2093208,4933,⋯,69,91,47,160,0.6312420,0.2983355,56.44913,NA,NA,NA
2,2012.917,0.06257839,0.3043835,0.1316908,0.0008046444,0.10657801,0.12409638,0.07945173,0.1904166,5568,⋯,88,107,59,154,0.6642066,0.2607626,57.58885,1.1397139,0.032964645,-0.037572860
3,2013.000,0.06896527,0.3465094,0.1351690,0.0006823642,0.09057575,0.08579316,0.08160333,0.1907017,5717,⋯,66,58,57,143,0.6630728,0.2654987,57.84830,0.2594500,-0.001133866,0.004736045
4,2013.083,0.06287183,0.3665349,0.1593783,0.0006558756,0.09518105,0.08433459,0.04941335,0.1816300,4670,⋯,68,53,26,131,0.6259205,0.2812960,57.38243,-0.4658647,-0.037152305,0.015797371
5,2013.167,0.07244608,0.3433993,0.1396045,0.0006720342,0.09523713,0.14539373,0.05562080,0.1476265,7253,⋯,77,119,36,100,0.6055626,0.3021492,56.12384,-1.2585885,-0.020357892,0.020853155
6,2013.250,0.06876135,0.3047955,0.1389688,0.0006749382,0.09800549,0.14669672,0.05030655,0.1917906,5964,⋯,65,111,33,151,0.6450331,0.2569536,58.05673,1.9328913,0.039470534,-0.045195536


In [3]:
# Load data on GDP growth and expectations
expectations <- read_excel("../../text_analysis/data/expectations.xlsx")
gdp <- read_excel("../../text_analysis/data/gdp.xlsx")
# Load more real-time indicators: 
words <- read_excel("../../text_analysis/data/google_trends_words.xlsx")
elec <- read_excel("../../text_analysis/data/electricity.xlsx")

In [4]:
# Merge
data <- expectations %>%
    full_join(gdp,by="month") %>%
    full_join(words,by="month") %>%
    full_join(elec,by="month") %>%
    mutate(month=as.numeric(as.yearmon(month))) %>%
    full_join(news_variables,by="month") 
head(data)

month,demand3m,econ3m,sector3m,emp3m,firm3m,econ1yr,sector1yr,production,sales,⋯,news_5,news_6,news_7,news_8,pos_dum,neg_dum,index,var_index,var_pos,var_neg
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2011.833,59.53177,57.52508,57.21477,53.17726,58.86288,60.50000,63.16667,55.94595,54.94881,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2011.917,56.85619,55.79470,56.64452,54.00000,58.83333,59.42761,61.48649,55.20833,56.33333,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2012.000,61.20401,57.33333,58.02676,54.50000,61.66667,62.16667,64.54849,53.15789,55.33333,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2012.083,63.75405,59.51613,61.48867,57.14286,63.31169,66.77419,68.12298,52.63158,56.20915,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2012.167,64.03785,63.53503,62.53968,57.30159,64.12698,70.79365,69.04762,58.85417,61.66134,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2012.250,64.47811,64.98316,61.82432,58.92256,63.25503,73.47973,71.79054,57.54190,57.16724,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [5]:
# Specify number of topics
k = 8

In [6]:
# Variable names
colnames(data)

[1] "month"               "demand3m"            "econ3m"             
 [4] "sector3m"            "emp3m"               "firm3m"             
 [7] "econ1yr"             "sector1yr"           "production"         
[10] "sales"               "demand"              "purchases"          
[13] "inventories"         "firmsituation"       "saleprice"          
[16] "inputprice"          "gdp"                 "gdp_nprim"          
[19] "gdp_nprim_wo_manuf"  "gdp_prim"            "mining"             
[22] "manufacturing"       "manufacturing_nprim" "electricity"        
[25] "construction"        "commerce"            "services"           
[28] "transport"           "restaurants"         "telecom"            
[31] "finserv"             "firmserv"            "publicadm"          
[34] "otherserv"           "totalserv"           "mayor"              
[37] "crecimiento"         "bien"                "importante"         
[40] "menor"               "riesgo"              "menos"              
[43] "bajo"                "mercado"             "presidente"         
[46] "gobierno"            "eeuu"                "empresa"            
[49] "dolar"               "proyecto"            "trabajo"            
[52] "desempleo"           "enfermo"             "var_elec"           
[55] "var_elecsm"          "gamma_1"             "gamma_2"            
[58] "gamma_3"             "gamma_4"             "gamma_5"            
[61] "gamma_6"             "gamma_7"             "gamma_8"            
[64] "count_1"             "count_2"             "count_3"            
[67] "count_4"             "count_5"             "count_6"            
[70] "count_7"             "count_8"             "news_1"             
[73] "news_2"              "news_3"              "news_4"             
[76] "news_5"              "news_6"              "news_7"             
[79] "news_8"              "pos_dum"             "neg_dum"            
[82] "index"               "var_index"           "var_pos"            
[85] "var_neg"

In [7]:
# Make lists
gammas <- which(colnames(data)=="gamma_1"):which(colnames(data)==paste("gamma",k,sep="_"))
list_gammas <- colnames(data)[gammas]
counts <- which(colnames(data)=="count_1"):which(colnames(data)==paste("news",k,sep="_"))
list_counts <- colnames(data)[counts]
sentiment <- which(colnames(data)=="pos_dum"):which(colnames(data)=="var_neg")
list_sentiment <- colnames(data)[sentiment]
exp <- which(colnames(data)=="demand3m"):which(colnames(data)=="inputprice")
list_exp <- colnames(data)[exp]
gdp_vars <- which(colnames(data)=="gdp"):which(colnames(data)=="totalserv")
list_gdp_vars <- colnames(data)[gdp_vars]
other_vars <- which(colnames(data)=="mayor"):which(colnames(data)=="var_elecsm")
list_other_vars <- colnames(data)[other_vars]

In [8]:
# Two functions: percentual variation and difference
percentvar <- function(x){
    ((x+lag(x,1))/(lag(x,12)+lag(x,13)) - 1)* 100 # To prevent Inf values, taking two-period averages.
}
diffvar <- function(x){
    (x - lag(x,12))
}

In [9]:
# Calculate percentual variation for variables in levels (counts of words and news)
list_var_counts <- paste("var",list_counts,sep="_")
var_counts_data <- as.data.frame(matrix(, nrow = nrow(data), ncol = length(list_counts)))
for (i in 1:length(list_counts)) {
    var_counts_data[,i] <- percentvar(data[,c(list_counts[[i]])])
    colnames(var_counts_data)[i] <- list_var_counts[i] 
}

In [10]:
# Special case
# Topic #4 is related to the COVID pandemic. Before 2020, the number of news that mostly covered that topic was 0. 
# Filling missings as 0 before 2020 and adding a dummy for post-COVID.
news_variables$news_4 
var_counts_data$var_news_4
var_counts_data$var_news_4 <- ifelse(is.na(var_counts_data$var_news_4) | !is.finite(var_counts_data$var_news_4) ,0 ,var_counts_data$var_news_4)
var_counts_data$covid <- data$month > 2020

[1]    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
[16]    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
[31]    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
[46]    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
[61]    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
[76]    0    0    1    0    0    0  838  849  820  840  719  790  745 1634 1543
[91] 1440 1517 1367 1221

[1]          NA          NA          NA          NA          NA          NA
  [7]          NA          NA          NA          NA          NA          NA
 [13]          NA          NA          NA          NA          NA          NA
 [19]          NA          NA          NA          NA          NA          NA
 [25]          NA         NaN         NaN         NaN         NaN         NaN
 [31]         NaN         NaN         NaN         NaN         NaN         NaN
 [37]         NaN         NaN         NaN         NaN         NaN         NaN
 [43]         NaN         NaN         NaN         NaN         NaN         NaN
 [49]         NaN         NaN         NaN         NaN         NaN         NaN
 [55]         NaN         NaN         NaN         NaN         NaN         NaN
 [61]         NaN         NaN         NaN         NaN         NaN         NaN
 [67]         NaN         NaN         NaN         NaN         NaN         NaN
 [73]         NaN         NaN         NaN         NaN         NaN         NaN
 [79]         NaN         NaN         NaN         NaN         NaN         NaN
 [85]         NaN         NaN         NaN         NaN         NaN         Inf
 [91]         Inf         NaN         NaN         Inf         Inf         Inf
 [97]         Inf         Inf         Inf         Inf         Inf 317600.0000
[103] 298200.0000         Inf         Inf    208.8305          NA

In [11]:
# Calculate difference for proportions (gammas)
list_var_gammas <- paste("var",list_gammas,sep="_")
var_gammas_data <- as.data.frame(matrix(, nrow = nrow(data), ncol = length(list_gammas)))
for (i in 1:length(list_gammas)) {
    var_gammas_data[,i] <- diffvar(data[,c(list_gammas[[i]])])
    colnames(var_gammas_data)[i] <- list_var_gammas[i] 
}

In [12]:
# Function: find the best time lag 
Find_Abs_Max_CCF<- function(a,b) {
    d <- ccf(a, b, plot = FALSE, lag.max = 3)
    cor = d$acf[,,1]
    abscor = abs(d$acf[,,1])
    lag = d$lag[,,1]
    res = data.frame(cor,lag)
    absres = data.frame(abscor,lag)
    absres_max = res[which.max(absres$abscor),]
    return(absres_max)
}  

In [13]:
# Identify the best lag for gammas
data_for_correl <- data %>%
filter(!is.na(gamma_1))
lags <- matrix(, length(c(gammas)), 2)
for (i in 1:length(c(gammas))) {
    if (i<=k) {
    max <- Find_Abs_Max_CCF(data_for_correl[,c(list_gammas[[i]])],data_for_correl$gdp)
      lags[i,1] <-max[[1]] 
      lags[i,2] <-max[[2]] 
        }
      }
lags

-0.2831300,-1
0.4415099,-3
0.4918664,0
-0.6477444,-1
0.3638346,0
0.2206982,-1
0.1948996,-3
0.4525459,-3


In [14]:
# Calculate lags for gammas and counts
num_lag <-1
list_l_gammas <- paste("l",list_gammas,sep="_")
list_l_counts <- paste("l",list_counts,sep="_")
l1 <- as.data.frame(matrix(, nrow = nrow(data), ncol = length(list_l_gammas)))
l2 <- as.data.frame(matrix(, nrow = nrow(data), ncol = length(list_l_counts)))
for (i in 1:length(list_gammas)) {
    l1[,i] <- lag(data[,c(list_gammas[[i]])], num_lag, na.pad = TRUE)
    colnames(l1)[i] <- list_l_gammas[i] 
}
for (i in 1:length(list_counts)) {
    l2[,i] <- lag(data[,c(list_counts[[i]])], num_lag, na.pad = TRUE)
    colnames(l2)[i] <- list_l_counts[i] 
}

In [15]:
# Calculate lags for sentiment variables
list_l_sentiment <- paste("l",list_sentiment,sep="_")
l3 <- as.data.frame(matrix(, nrow = nrow(data), ncol = length(list_l_sentiment)))
for (i in 1:length(list_sentiment)) {
    l3[,i] <- lag(data[,c(sentiment[[i]])], num_lag, na.pad = TRUE)
    colnames(l3)[i] <- list_l_sentiment[i] 
}

In [16]:
# Merge with original dataset
all_data <- cbind(data,var_gammas_data,var_counts_data,l1,l2,l3)

In [17]:
#Calculate lags for variation in gammas/counts
list_l_var_gammas <- paste("l",list_var_gammas,sep="_")
list_l_var_counts <- paste("l",list_var_counts,sep="_")

In [18]:
l4 <- as.data.frame(matrix(, nrow = nrow(data), ncol = length(list_l_gammas)))
l5 <- as.data.frame(matrix(, nrow = nrow(data), ncol = length(list_l_counts)))
for (i in 1:length(list_var_gammas)) {
    l4[,i] <- lag(all_data[,c(list_var_gammas[[i]])], num_lag, na.pad = TRUE)
    colnames(l4)[i] <- list_l_var_gammas[i] 
}
for (i in 1:length(list_var_counts)) {
    l5[,i] <- lag(all_data[,c(list_var_counts[[i]])], num_lag, na.pad = TRUE)
    colnames(l5)[i] <- list_l_var_counts[i] 
}

In [19]:
# Extract month, get month dummies
month_var <- tibble(m=as.factor(format(as.Date(as.yearmon(data$month)),"%m")))
month_dummies <- dummy_cols(month_var,select_columns="m")
# Extract year, get year dummies
year_var <- tibble(y=as.factor(format(as.Date(as.yearmon(data$month)),"%y")))
year_dummies <- dummy_cols(year_var,select_columns="y")

In [20]:
# Combine all variables
all_data <- cbind(all_data,l4,l5,month_dummies,year_dummies)
head(all_data)
colnames(all_data)

,month,demand3m,econ3m,sector3m,emp3m,firm3m,econ1yr,sector1yr,production,sales,⋯,y_11,y_12,y_13,y_14,y_15,y_16,y_17,y_18,y_19,y_20
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,2011.833,59.53177,57.52508,57.21477,53.17726,58.86288,60.50000,63.16667,55.94595,54.94881,⋯,1,0,0,0,0,0,0,0,0,0
2,2011.917,56.85619,55.79470,56.64452,54.00000,58.83333,59.42761,61.48649,55.20833,56.33333,⋯,1,0,0,0,0,0,0,0,0,0
3,2012.000,61.20401,57.33333,58.02676,54.50000,61.66667,62.16667,64.54849,53.15789,55.33333,⋯,0,1,0,0,0,0,0,0,0,0
4,2012.083,63.75405,59.51613,61.48867,57.14286,63.31169,66.77419,68.12298,52.63158,56.20915,⋯,0,1,0,0,0,0,0,0,0,0
5,2012.167,64.03785,63.53503,62.53968,57.30159,64.12698,70.79365,69.04762,58.85417,61.66134,⋯,0,1,0,0,0,0,0,0,0,0
6,2012.250,64.47811,64.98316,61.82432,58.92256,63.25503,73.47973,71.79054,57.54190,57.16724,⋯,0,1,0,0,0,0,0,0,0,0


[1] "month"               "demand3m"            "econ3m"             
  [4] "sector3m"            "emp3m"               "firm3m"             
  [7] "econ1yr"             "sector1yr"           "production"         
 [10] "sales"               "demand"              "purchases"          
 [13] "inventories"         "firmsituation"       "saleprice"          
 [16] "inputprice"          "gdp"                 "gdp_nprim"          
 [19] "gdp_nprim_wo_manuf"  "gdp_prim"            "mining"             
 [22] "manufacturing"       "manufacturing_nprim" "electricity"        
 [25] "construction"        "commerce"            "services"           
 [28] "transport"           "restaurants"         "telecom"            
 [31] "finserv"             "firmserv"            "publicadm"          
 [34] "otherserv"           "totalserv"           "mayor"              
 [37] "crecimiento"         "bien"                "importante"         
 [40] "menor"               "riesgo"              "menos"              
 [43] "bajo"                "mercado"             "presidente"         
 [46] "gobierno"            "eeuu"                "empresa"            
 [49] "dolar"               "proyecto"            "trabajo"            
 [52] "desempleo"           "enfermo"             "var_elec"           
 [55] "var_elecsm"          "gamma_1"             "gamma_2"            
 [58] "gamma_3"             "gamma_4"             "gamma_5"            
 [61] "gamma_6"             "gamma_7"             "gamma_8"            
 [64] "count_1"             "count_2"             "count_3"            
 [67] "count_4"             "count_5"             "count_6"            
 [70] "count_7"             "count_8"             "news_1"             
 [73] "news_2"              "news_3"              "news_4"             
 [76] "news_5"              "news_6"              "news_7"             
 [79] "news_8"              "pos_dum"             "neg_dum"            
 [82] "index"               "var_index"           "var_pos"            
 [85] "var_neg"             "var_gamma_1"         "var_gamma_2"        
 [88] "var_gamma_3"         "var_gamma_4"         "var_gamma_5"        
 [91] "var_gamma_6"         "var_gamma_7"         "var_gamma_8"        
 [94] "var_count_1"         "var_count_2"         "var_count_3"        
 [97] "var_count_4"         "var_count_5"         "var_count_6"        
[100] "var_count_7"         "var_count_8"         "var_news_1"         
[103] "var_news_2"          "var_news_3"          "var_news_4"         
[106] "var_news_5"          "var_news_6"          "var_news_7"         
[109] "var_news_8"          "covid"               "l_gamma_1"          
[112] "l_gamma_2"           "l_gamma_3"           "l_gamma_4"          
[115] "l_gamma_5"           "l_gamma_6"           "l_gamma_7"          
[118] "l_gamma_8"           "l_count_1"           "l_count_2"          
[121] "l_count_3"           "l_count_4"           "l_count_5"          
[124] "l_count_6"           "l_count_7"           "l_count_8"          
[127] "l_news_1"            "l_news_2"            "l_news_3"           
[130] "l_news_4"            "l_news_5"            "l_news_6"           
[133] "l_news_7"            "l_news_8"            "l_pos_dum"          
[136] "l_neg_dum"           "l_index"             "l_var_index"        
[139] "l_var_pos"           "l_var_neg"           "l_var_gamma_1"      
[142] "l_var_gamma_2"       "l_var_gamma_3"       "l_var_gamma_4"      
[145] "l_var_gamma_5"       "l_var_gamma_6"       "l_var_gamma_7"      
[148] "l_var_gamma_8"       "l_var_count_1"       "l_var_count_2"      
[151] "l_var_count_3"       "l_var_count_4"       "l_var_count_5"      
[154] "l_var_count_6"       "l_var_count_7"       "l_var_count_8"      
[157] "l_var_news_1"        "l_var_news_2"        "l_var_news_3"       
[160] "l_var_news_4"        "l_var_news_5"        "l_var_news_6"       
[163] "l_var_news_7"        "l_var_news_8"        "m"                  
[166] "m_01"                "m_02"        

In [21]:
# Get column numbers
topic_var <- match(c(list_var_gammas,list_var_counts),names(all_data))
topic_lags <- match(c(list_l_gammas,list_l_counts,list_l_sentiment),names(all_data))
time_dummies <- which(colnames(all_data)=="m_01"):which(colnames(all_data)=="y_20")
topic_var_lags <- match(c(list_l_var_gammas,list_l_var_counts),names(all_data))

## Getting data in the right shape for Lasso

In [22]:
# Remove missings
all_data <- all_data %>%
    filter(complete.cases(.)) 

In [23]:
# Error function
get_error <- function(yreal,ypred){
    error <- (mean((yreal - ypred)^2))^0.5
    return(error)
  }

## Lasso

In [24]:
# Parameters
n <- nrow(all_data)
num_outputs <- length(gdp_vars)
window_val <- 48
horizon_val <- 12
max_T <- nrow(all_data) - window_val - horizon_val - num_lag

In [25]:
# Caret function and parameters
feature_formula <- paste(c(list_gammas,list_counts,list_sentiment,list_l_gammas,list_l_counts,list_l_sentiment,list_exp,list_var_gammas,list_var_counts,list_l_var_gammas,list_l_var_counts,'covid',list_other_vars), collapse = " + ")
myTimeControl <- trainControl(method = "timeslice", initialWindow = window_val, horizon = horizon_val,fixedWindow = FALSE, allowParallel = TRUE, savePredictions = "all")

In [26]:
# Matrices to be filled
rolling_window <- data.frame(rw= matrix(,nrow=n,ncol=num_outputs))
prediction <- data.frame(pred = matrix(,nrow=n,ncol=num_outputs))
direction_change <- data.frame(matrix(,nrow=n,ncol=num_outputs))
error <- data.frame(matrix(,nrow=1,ncol=num_outputs))
importance_list <- list()

In [27]:
# Prediction
suppressWarnings({ # Models where the prediction is close to a constant give missing R2. Details in the next chunk of code.
    for (i in 1:num_outputs) { # For all outputs 
        glmnet_formula   <- paste(list_gdp_vars[i], feature_formula, sep = " ~ ")
        grid <- expand.grid(alpha = seq(0,1, length = 3), lambda = seq(0,1, length = 10))
        for (T in max_T:2) { # Get rolling window predictions for different time horizons
              glmnet_model <- train(as.formula(glmnet_formula),data = all_data[1:(n-T),],method = "glmnet",tuneGrid=grid,trControl = myTimeControl)
              plot(glmnet_model)
              output_temp <- as.data.frame(predict(glmnet_model,newdata=all_data))
              rolling_window[(n-T+1),i] <- output_temp[(n-T+1),]
              direction_change[(n-T+1),i] <- ifelse(output_temp[(n-T+1),] > output_temp[(n-T),],1,ifelse(output_temp[(n-T+1),] < output_temp[(n-T),],-1,0))
              if (T==2) { # Input the prediction for the period of interest
                prediction[,i] <- output_temp
                rolling_window[n,i] <- output_temp[n,]
                direction_change[n,i] <- ifelse(output_temp[n,] > output_temp[(n-1),],1,ifelse(output_temp[n,] < output_temp[(n-1),],-1,0))
                importance_list[[i]] <- varImp(glmnet_model,scale=T)
              }
        }
    }
})

In [28]:
# The model gives warning when R square is missing (predictions are the same, have zero variance because the model is predicting a value close to a constant). 
# Source: https://stackoverflow.com/questions/51548255/caret-there-were-missing-values-in-resampled-performance-measures
# Tends to happen when alpha==1 & lambda==1. These models aren't selected. For further exploration, run the code below:
# glmnet_model$results
# glmnet_model$pred %>%
#   filter(alpha==1 & lambda == 1) -> for_r2
# var(for_r2$pred)
# unique(for_r2$pred)
# caret::R2(for_r2$obs, for_r2$pred)

In [29]:
# RMSE (rolling window prediction vs. true values)
for (i in 1:num_outputs) {
  error[,i] <- get_error(rolling_window[(n-max_T+1):n,i],all_data[(n-max_T+1):n,gdp_vars[i]])
}
error

X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6.498884,6.818794,7.424312,128.8535,207.2914,1160.046,477.0725,1.589839,276.762,13.76777,7.334962,14.43189,23.97446,113.2871,11.12626,14.42026,0.4072622,7.996825,6.466991


In [30]:
# We want to know if the model is good at predicting the direction of change (ie, whether the growth rate next month is higher or lower than the previous month). 
## Get direction of change in real data
direction_change_true <- data.frame(matrix(,nrow=n,ncol=num_outputs))
for (i in 1:num_outputs) {
  direction_change_true[,i] <- ifelse(all_data[,gdp_vars[i]] > lag(all_data[,gdp_vars[i]]),1,ifelse(all_data[,gdp_vars[i]] < lag(all_data[,gdp_vars[i]]),-1,0))
}
## Compare 
match <- data.frame(matrix(,nrow=1,ncol=num_outputs))
for (i in 1:num_outputs) {
    match[,i] <- mean(direction_change_true[,i] == direction_change[,i],na.rm=TRUE)
}
match

X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.8947368,0.7894737,0.6842105,0.6842105,0.6842105,0.7368421,0.8421053,0.9473684,0.6315789,0.6315789,0.4210526,0.4736842,0.2105263,0.4210526,0.4210526,0.8421053,0.4210526,0.5263158,0.3157895


## Plots

In [31]:
# Get time series ready (predictions + outputs)
data_for_plots <- all_data %>%
    cbind(prediction,rolling_window) %>%
    filter(complete.cases(.))
prediction_series <- which(colnames(data_for_plots)=="pred.1"):which(colnames(data_for_plots)=="pred.19")
rw_series <- which(colnames(data_for_plots)=="rw.1"):which(colnames(data_for_plots)=="rw.19")

In [32]:
# Label GDP variables, prediction series, and rolling window forecasts
gdp_vars_label <- paste(" ",paste(capitalize(gsub("_", " ", list_gdp_vars,fixed = TRUE)),"(Var. %)",sep=" "),sep="")
prediction_vars_label <- paste(" ","Series prediction",sep="")
rw_vars_label <- paste("","RW prediction",sep="")

In [33]:
# Storing plots
plot_list = list() 
fname <-paste("../tables_plots/plots.pdf")
pdf(fname)
#Plot
for (i in 1:num_outputs) { 
    # Correlation label
    correl = paste("Cor.: ", round(cor(data_for_plots[,gdp_vars[i]],data_for_plots[,prediction_series[i]]),2) ,sep="")
    # RMSE label
    rmse_rw = paste("RMSE (RW): ", round(error[i],2),sep="")
    # Match label
    match_rw = paste("Match (RW): ",round(match[i],2),sep="")
    # Final label
    title_label= paste(paste(paste(correl,match_rw,sep=","),rmse_rw,sep=","))
    # Plot
    p <- ggplot() + 
    theme_light() +
    geom_line(data=data_for_plots,aes(x=month,y = data_for_plots[,gdp_vars[i]],color=gdp_vars_label[i])) +
    geom_line(data=data_for_plots,aes(x=month,y = data_for_plots[,prediction_series[i]],colour=prediction_vars_label)) +
    geom_point(data=data_for_plots,aes(x=month,y = data_for_plots[,rw_series[i]],colour=rw_vars_label)) +
    scale_colour_manual(values = c("DarkBlue","Red","DarkOrange")) +
    labs(y = "Var%", x = "Month", colour = title_label) +
    theme(legend.position = c(0.25, 0.16)) +
    theme(legend.key.size = unit(0.5, "cm"),legend.title = element_text(size=9)) +
    theme(legend.background=element_blank()) 
    plot_list[[i]] = p
    print(plot_list[[i]])
}
dev.off()

pdf 
  2

In [34]:
# Storing variable importance plots
plot_list = list() 
fname <-paste("../tables_plots/var_imp.pdf")
pdf(fname)
#Plot
for (i in 1:num_outputs) { 
    print(plot(importance_list[[i]],top=35))
}
dev.off()

pdf 
  2

In [35]:
# Export
results <- rbind(error,match)
colnames(results) <- list_gdp_vars
predictions <- rolling_window[c(n-1,n),]
colnames(predictions) <- list_gdp_vars
results <- rbind(results,predictions)
rownames(results) <- c("RMSE","Match","August prediction","September prediction")
write.csv(results,"../tables_plots/results.csv")
results

,gdp,gdp_nprim,gdp_nprim_wo_manuf,gdp_prim,mining,manufacturing,manufacturing_nprim,electricity,construction,commerce,services,transport,restaurants,telecom,finserv,firmserv,publicadm,otherserv,totalserv
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RMSE,6.4988840,6.8187936,7.4243124,128.8534682,207.2913606,1160.0461863,477.0725111,1.5898391,276.7619755,13.7677660,7.3349619,14.4318892,23.9744575,113.2871303,11.1262556,14.4202607,0.4072622,7.9968254,6.4669908
Match,0.8947368,0.7894737,0.6842105,0.6842105,0.6842105,0.7368421,0.8421053,0.9473684,0.6315789,0.6315789,0.4210526,0.4736842,0.2105263,0.4210526,0.4210526,0.8421053,0.4210526,0.5263158,0.3157895
August prediction,-8.9779005,-13.0250825,-13.6591134,-5.9727509,-10.0008849,-14.6934265,-12.2311891,-4.7842269,-24.7206327,-17.1609782,-9.7057895,-24.0924983,-41.6958179,5.5263431,6.9139977,-18.1361588,4.3731179,-8.9816969,-11.0067446
September prediction,-6.5328759,-10.9502713,-12.3297408,-3.5410206,-7.0075313,-12.1210930,-9.2114652,-2.1214094,-18.9622415,-15.6946953,-7.4664317,-19.9407839,-37.5038570,5.7070007,7.5170498,-13.6905908,4.4554179,-7.6339274,-9.0815766
